In [212]:
# Implementation Of Regression

import pandas as pd
import torch 
import numpy as np
import tensorflow as tf

In [213]:
veh = pd.read_csv("vehicle_csv.csv")
veh.head()

,COMPACTNESS,CIRCULARITY,DISTANCE_CIRCULARITY,RADIUS_RATIO,PR.AXIS_ASPECT_RATIO,MAX.LENGTH_ASPECT_RATIO,SCATTER_RATIO,ELONGATEDNESS,PR.AXIS_RECTANGULARITY,MAX.LENGTH_RECTANGULARITY,SCALED_VARIANCE_MAJOR,SCALED_VARIANCE_MINOR,SCALED_RADIUS_OF_GYRATION,SKEWNESS_ABOUT_MAJOR,SKEWNESS_ABOUT_MINOR,KURTOSIS_ABOUT_MAJOR,KURTOSIS_ABOUT_MINOR,HOLLOWS_RATIO,Class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus


In [214]:
veh["Class"].unique()

array(['van', 'saab', 'bus', 'opel'], dtype=object)

In [215]:
X = torch.tensor(veh.drop("Class", axis=1).values, dtype=torch.float)
Y = torch.tensor(
    [0 if vty == "van" else 1 if vty == "saab" else 2 if vty == "bus" else 3 if vty == "opel" else 4  for vty in veh["Class"]], 
    dtype=torch.long
)

print(X.shape,Y.shape)

torch.Size([846, 18]) torch.Size([846])


In [216]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

In [217]:
samples, features = X_train.shape
print(features)

18


In [218]:
X_train = torch.tensor(X_train,dtype=float)
X_test = torch.tensor(X_test,dtype=float)
Y_train = torch.tensor(Y_train,dtype=float)
Y_test = torch.tensor(Y_test,dtype=float)

C:\Users\dllab\AppData\Local\Temp\ipykernel_13760\80705368.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train,dtype=float)
C:\Users\dllab\AppData\Local\Temp\ipykernel_13760\80705368.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test,dtype=float)
C:\Users\dllab\AppData\Local\Temp\ipykernel_13760\80705368.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_train = torch.tensor(Y_train,dtype=float)
C:\Users\dllab\AppData\Local\Temp\ipykernel_13760\80705368.py:4: UserWarning: To 

In [219]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

torch.Size([676, 18]) torch.Size([170, 18]) torch.Size([676]) torch.Size([170])


In [220]:
from torch import nn

class regression(nn.Module):
    def __init__(self):
        super(regression, self).__init__()
        self.first_layer = nn.Linear(features,34)
        self.second_layer = nn.Linear(34, 15)
        self.final_layer = nn.Linear(15,1)
        self.relu = nn.ReLU()

    def forward(self,X_batch):
        layer_out = self.relu(self.first_layer(X_batch))
        layer_out = self.relu(self.second_layer(layer_out))
        return self.final_layer(layer_out)

In [221]:
X_train = X_train.to(torch.float32)
X_train.dtype

torch.float32

In [222]:
regressor = regression()
preds = regressor(X_train[:5])
print(preds)

tensor([[33.3838],
        [43.8678],
        [46.5394],
        [45.1727],
        [50.2820]], grad_fn=<AddmmBackward0>)


In [223]:
def TrainModel(model, loss_func, optimizer, X, Y, epochs=1000):
    for i in range(epochs):
        preds = model(X)
        preds = preds.to(torch.float)
        loss = loss_func(preds.ravel(), Y)
        loss = loss.to(torch.float)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("MSE : {:.2f}".format(loss))

In [224]:
from torch.optim import SGD

e = 1000
learning_rate = torch.tensor(1/1e3)
learning_rate = learning_rate.to(torch.float)

regressor = regression()
mse_loss = nn.MSELoss()
optimizer = SGD(params=regressor.parameters(), lr=learning_rate)

X_train = X_train.to(torch.float)
Y_train = Y_train.to(torch.float)

In [225]:
TrainModel(regressor, mse_loss, optimizer, X_train, Y_train, epochs = e)

MSE : 175.60
MSE : 4.35
MSE : 3.29
MSE : 2.60
MSE : 2.15
MSE : 1.85
MSE : 1.65
MSE : 1.52
MSE : 1.43
MSE : 1.37


In [226]:
X_test = X_test.to(torch.float)

In [227]:
train_preds = regressor(X_train)
train_preds[:5]

tensor([[1.2703],
        [1.2703],
        [1.2703],
        [1.2703],
        [1.2703]], grad_fn=<SliceBackward0>)

In [228]:
test_preds = regressor(X_test) 
test_preds[:5]

tensor([[1.2703],
        [1.2703],
        [1.2703],
        [1.2703],
        [1.2703]], grad_fn=<SliceBackward0>)

In [229]:
from sklearn.metrics import r2_score

print("Train R^2 Score : {:.2f}".format(r2_score(train_preds.detach().numpy().squeeze(), Y_train.detach().numpy())))

Train R^2 Score : 0.00
